# google drive

因为尝试了官方文档的方法不成功，所以使用[pyDrive](https://github.com/gsuitedevs/PyDrive)尝试一下。

首先安装：

```bash
pip install PyDrive
```

然后从Google API Console 下载client_secrets.json 文件，搜索google console，点击进入google cloud console，如果没有google cloud console，那就需要从头开始申请一个，这部分这里就暂不介绍了。选择“API和服务”后，点击“库”，进入库页面，搜索drive，启用该API。

然后还选择“API和服务”，然后点击“凭据”进入页面，点击“创建凭据”，选择“OAuth客户端ID”，创建一个“OAuth 2.0 客户端 ID”，中间会让用户开启一个“OAuth同意屏幕”，随便输入一个项目名称开启即可，然后在回到“创建凭据”，创建一个OAuth 2.0 客户端 ID。这时候就有一个OAuth 2.0 客户端 ID条目出现了，点击右边下载按键即可下载client_secrets.json。自己在使用过程中最好生成一个本地凭证，这样不用总是需要远程认证，方法参考了：[Automating pydrive verification process](https://stackoverflow.com/questions/24419188/automating-pydrive-verification-process/24542604#24542604)的高票回答，即通过下面的执行语句，利用gauth.SaveCredentialsFile("mycreds.txt")生成一个本地的credits文件，再后面再使用的话可以通过如下代码实现获取凭证：

```python
gauth.LoadCredentialsFile(client_secrets_file)
```

这里就不对这种方法进行实例展示了。继续其他对dirve的操作：

In [14]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

gauth = GoogleAuth()
gauth.LocalWebserverAuth()
gauth.SaveCredentialsFile("mycreds.txt")
drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=974421890803-k4ls1nquvh33hlg8g1ttaab5lu1nq5a7.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


接下来就可以很简单地实现文件的上传和下载了。首先上传一个文件：

In [2]:
file1 = drive.CreateFile({'title': 'Hello.txt'})
file1.SetContentString('Hello')
file1.Upload() # Files.insert()

然后可以直接修改该文件名称：

In [3]:
file1['title'] = 'HelloWorld.txt'  # Change title of the file
file1.Upload() # Files.patch()

接下来可以试试往文件中增加内容。

In [4]:
content = file1.GetContentString()  # 'Hello'
file1.SetContentString(content+' World!')  # 'Hello World!'
file1.Upload() # Files.update()

下载文件使用GetContentFile(filename)或者 GetContentString().

In [5]:
file2 = drive.CreateFile({'id': file1['id']})
print('Downloading file %s from Google Drive' % file2['title'])
type(file1['id'])

str

Save Drive file as a local file，参数是本地文件的名称

In [7]:
file2.GetContentFile('HelloWorld.txt')  

还可以查询指定目录下的文件

In [8]:
# Auto-iterate through all files that matches this query
file_list1 = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file1 in file_list1:
    print('title: %s, id: %s' % (file1['title'], file1['id']))
    if file1['title']=='test':
        file1_id=str(file1['id'])
        #  列出某个文件夹下的文件        
        print(file1_id)
        file_list = drive.ListFile({'q': "'"+file1_id+"' in parents and trashed=false"}).GetList()
        for file in file_list:
            print('title: %s, id: %s' % (file['title'], file['id']))

title: HelloWorld.txt, id: 1g4mT39PcEQEs17oVOIIKRRQFLH0G4Vuc
title: refactor the codes, id: 1sSuNTbyftvZmXFthxtweBPGBBQH8IEDyABiKg4M6bG8
title: WRData, id: 1Ra-uA7lT_HBvPI2afQkYxXxjkWOuRRuQ
title: test, id: 1jhJpA9qKSmY0bi3jOvMl9FaGLcKIfN2I
1jhJpA9qKSmY0bi3jOvMl9FaGLcKIfN2I
title: HelloWorld.txt, id: 1HJOkPe0dlrU_97T6h6QxvNmYGhbihVdP
title: foo.txt, id: 1RdQHHeHsHakuyCLzguC7Y36R7DxUIzF0
title: daymet, id: 1bFV4R6FW5WUlUoVkLsO25YhUzeeFvHCA
title: RS, id: 1G1a9rbPsHnbJ6xhlznKB0U_UrxwXrWw0
title: drive, id: 1-4LOn3r-drroBNMYwEA7HBMZNmgW5pM1
title: Colab Notebooks, id: 1dtSHtNnHBRAJo5xjBM2CEmwYETU3BovM
title: Life, id: 1Oml-Ia5ricBjFNT2yV-m1D9Nhoch0Ohy
title: Research, id: 1UTOQVqsqqzbOBGRW7KeLs91GDDS1JT7g


从上可知，下载一个文件需要指定其id。如何查看某个文件夹下的id呢？，需要在ListFIle函数中使用某个文件夹的id，这样可以如下列出文件夹下的文件。

In [9]:
# Auto-iterate through all files that matches this query
file_list = drive.ListFile({'q': "'1jhJpA9qKSmY0bi3jOvMl9FaGLcKIfN2I' in parents and trashed=false"}).GetList()
for file in file_list:
    print('title: %s, id: %s' % (file['title'], file['id']))

title: HelloWorld.txt, id: 1HJOkPe0dlrU_97T6h6QxvNmYGhbihVdP
title: foo.txt, id: 1RdQHHeHsHakuyCLzguC7Y36R7DxUIzF0


现在指定下载一个文件

In [10]:
file3 = drive.CreateFile({'id': '1HJOkPe0dlrU_97T6h6QxvNmYGhbihVdP'})
print('Downloading file %s from Google Drive' % file3['title'])
file3.GetContentFile(file3['title'])  

批量下载文件可使用下列代码：

In [12]:
# Auto-iterate through all files that matches this query
import os
file_list = drive.ListFile({'q': "'1jhJpA9qKSmY0bi3jOvMl9FaGLcKIfN2I' in parents and trashed=false"}).GetList()
for file in file_list:
    print('title: %s, id: %s' % (file['title'], file['id']))
    file_dl= drive.CreateFile({'id':file['id']})
    print('Downloading file %s from Google Drive' % file_dl['title'])
    # 下载到指定文件夹    
    directory="./test"
    file_dl.GetContentFile(os.path.join(directory,file_dl['title']))  
print('Downloading file finished')

title: HelloWorld.txt, id: 1HJOkPe0dlrU_97T6h6QxvNmYGhbihVdP
title: foo.txt, id: 1RdQHHeHsHakuyCLzguC7Y36R7DxUIzF0


以上就是简单利用PyDrive进行文件下载的方式。